In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.metrics import classification_report, confusion_matrix

import tensorflow as tf
import keras
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import applications
from keras.models import Sequential, load_model
from keras.layers import Conv2D, MaxPooling2D, GlobalAveragePooling2D, Flatten, Dense, Dropout
from keras.preprocessing import image

import warnings
warnings.filterwarnings('ignore')

In [2]:
train_image='D:/Documents/Capstone/Project -2/Dataset/blast_VS_non-blast/train'

In [3]:
train_datagen=ImageDataGenerator(rescale=1./255,
                                 zoom_range=0.2,
                                 shear_range=0.1,
                                vertical_flip=False)
training_set=train_datagen.flow_from_directory(train_image,
                                              target_size=(224,224),
                                              class_mode='categorical',
                                              batch_size=5)

Found 2366 images belonging to 2 classes.


In [4]:
test_datagen = ImageDataGenerator(rescale = 1./255,
                                 zoom_range=0.2)

test_set=train_datagen.flow_from_directory('D:/Documents/Capstone/Project -2/Dataset/bacterial_leaf_blight_Vs_non-bacterial_leaf_blight/test',
                                              target_size=(224,224),
                                              class_mode='categorical',
                                              batch_size=5)

Found 422 images belonging to 2 classes.


In [10]:
base_model = tf.keras.applications.InceptionResNetV2(
                     include_top=False,
                     weights='imagenet',
                     input_shape=(224,224,3)
                     )
 
base_model.trainable=False
model = tf.keras.Sequential([
        base_model,
        tf.keras.layers.BatchNormalization(renorm=True),
        tf.keras.layers.GlobalAveragePooling2D(),
        tf.keras.layers.Dense(2, activation='softmax')
    ])

In [11]:
model.compile(optimizer='Adam',loss='categorical_crossentropy',metrics=['accuracy'])
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 inception_resnet_v2 (Functi  (None, 5, 5, 1536)       54336736  
 onal)                                                           
                                                                 
 batch_normalization_406 (Ba  (None, 5, 5, 1536)       10752     
 tchNormalization)                                               
                                                                 
 global_average_pooling2d (G  (None, 1536)             0         
 lobalAveragePooling2D)                                          
                                                                 
 dense_1 (Dense)             (None, 2)                 3074      
                                                                 
Total params: 54,350,562
Trainable params: 6,146
Non-trainable params: 54,344,416
______________________________________

In [12]:
early = tf.keras.callbacks.EarlyStopping(patience=10, min_delta=0.001, restore_best_weights=True)

In [13]:
batch_size=32
STEP_SIZE_TRAIN = training_set.n//training_set.batch_size
STEP_SIZE_VALID = test_set.n//test_set.batch_size

# fit model
history = model.fit(training_set, steps_per_epoch=STEP_SIZE_TRAIN,
                    validation_data=test_set,
                    validation_steps=STEP_SIZE_VALID,
                    epochs=10,
                    callbacks=[early])

Epoch 1/10
473/473 [==============================] - 1008s 2s/step - loss: 0.2673 - accuracy: 0.8958 - val_loss: 1.4773 - val_accuracy: 0.5143
Epoch 2/10
473/473 [==============================] - 1925s 4s/step - loss: 0.1886 - accuracy: 0.9339 - val_loss: 1.7610 - val_accuracy: 0.5024
Epoch 3/10
473/473 [==============================] - 2083s 4s/step - loss: 0.1712 - accuracy: 0.9390 - val_loss: 2.1768 - val_accuracy: 0.5024
Epoch 4/10
473/473 [==============================] - 2149s 5s/step - loss: 0.1402 - accuracy: 0.9475 - val_loss: 1.9461 - val_accuracy: 0.5167
Epoch 5/10
473/473 [==============================] - 1986s 4s/step - loss: 0.1347 - accuracy: 0.9526 - val_loss: 2.1713 - val_accuracy: 0.4786
Epoch 6/10
473/473 [==============================] - 1805s 4s/step - loss: 0.1348 - accuracy: 0.9492 - val_loss: 1.9923 - val_accuracy: 0.4833
Epoch 7/10
473/473 [==============================] - 1834s 4s/step - loss: 0.1078 - accuracy: 0.9572 - val_loss: 2.5468 - val_accuracy: